In [54]:
from collections import deque
from keras.layers import Dense, Dropout
import gym
from keras.models import Sequential 
from keras.optimizers import Adam
import numpy as np
import random

In [55]:
# Deep Q-learning Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='tanh'))
        model.add(Dense(24, activation='tanh'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, min(batch_size,len(self.memory)))
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
              target = reward + self.gamma * \
                       np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [56]:
env = gym.make('CartPole-v0')

In [57]:
env.observation_space.sample()

array([ 1.6736554e+00, -2.4160252e+38,  3.3315143e-01,  2.1019791e+38],
      dtype=float32)

In [59]:
if __name__ == "__main__":
    # initialize gym environment and the agent
    env = gym.make('CartPole-v0')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size,action_size)
    episodes = 5000
    # Iterate the game
    for e in range(episodes):
        # reset state in the beginning of each game
        state = env.reset()
        state = np.reshape(state, [1, 4])
        # time_t represents each frame of the game
        # Our goal is to keep the pole upright as long as possible until score of 500
        # the more time_t the more score
        for time_t in range(500):
            # turn this on if you want to render
            env.render()
            # Decide action
            action = agent.act(state)
            # Advance the game to the next frame based on the action.
            # Reward is 1 for every frame the pole survived
            next_state, reward, done, _ = env.step(action)
            next_state = np.reshape(next_state, [1, 4])
            # Remember the previous state, action, reward, and done
            agent.remember(state, action, reward, next_state, done)
            # make next_state the new current state for the next frame.
            state = next_state
            # done becomes True when the game ends
            # ex) The agent drops the pole
            if done:
                # print the score and break out of the loop
                print("episode: {}/{}, score: {}"
                      .format(e, episodes, time_t))
                break
        # train the agent with the experience of the episode
        agent.replay(32)

episode: 0/5000, score: 25
episode: 1/5000, score: 87
episode: 2/5000, score: 43
episode: 3/5000, score: 38
episode: 4/5000, score: 12
episode: 5/5000, score: 16
episode: 6/5000, score: 9
episode: 7/5000, score: 10
episode: 8/5000, score: 14
episode: 9/5000, score: 40
episode: 10/5000, score: 21
episode: 11/5000, score: 24
episode: 12/5000, score: 13
episode: 13/5000, score: 21
episode: 14/5000, score: 26
episode: 15/5000, score: 32
episode: 16/5000, score: 12
episode: 17/5000, score: 24
episode: 18/5000, score: 23
episode: 19/5000, score: 32
episode: 20/5000, score: 28
episode: 21/5000, score: 28
episode: 22/5000, score: 16
episode: 23/5000, score: 32
episode: 24/5000, score: 59
episode: 25/5000, score: 34
episode: 26/5000, score: 16
episode: 27/5000, score: 11
episode: 28/5000, score: 25
episode: 29/5000, score: 19
episode: 30/5000, score: 19
episode: 31/5000, score: 18
episode: 32/5000, score: 12
episode: 33/5000, score: 17
episode: 34/5000, score: 21
episode: 35/5000, score: 22
epi

KeyboardInterrupt: 

In [61]:
env.close()